# **Trabalho Prático 1 - Geometria Computacional**


### **Universidade Federal de Minas Gerais - Instituto de Ciências Exatas**

#### Departamento de Ciência da Computação

DCC207 - Algoritmos 2

Prof. Renato Vimieiro


  
**Alunos:** Daniel Barbosa, Frederico Baker e Victor Prates

O presente trabalho visa a aplicação de algoritmos de geometria computacional para solucionar um problema de classificação em aprendizado de máquina supervisionado. Em especial, o problema de classificação a ser abordado se refere à triagem de pacientes em um serviço de saúde. A meta é criar um modelo de classificação usando técnicas como envoltória convexa e detecção de interseções entre segmentos.

No diretório ./models foram implementadas estruturas para representar pontos e segmentos, além das seguintes operações:
* Determinar a orientação relativa de dois segmentos
* Determinar se um ponto está sobre um segmento
* Determinar se dois segmentos se interceptam

In [34]:
from models.models import Point
from models.models import Segment

# 1. Envoltória convexa

# 2. Varredura linear

Aqui, vamos implementar um algoritmo que verifica se algum segment de uma envoltória A intercepta algum segmento de uma envoltória B.

Se há pelo o menos uma interceptação (ou seja, os dados não são linearmente separáveis), o algoritmo deve retornar que não é possível criar um modelo linear para tal conjunto de dados.

Se não há interceptação (ou seja, os dados são linearmente separáveis), o algoritmo deve retornar o modelo linear para tal conjunto de dados.

Primeiro, vamos tratar de um caso específico:

- Todos os segmentos de uma envoltória convexa se intersectam. Por isso, o algoritmo que impementaremos desconsiderará interseções que ocorrem exatamente na extremidade de um segmento com a extremidade de outro segmento.
    
    - Isso não atrapalhará na separabilidade das envoltórias, pois, se uma extremidade de envoltória intercepta uma extremidade da outra envoltória, essa extremidade em comum das envoltórias será exatamente o ponto pelo qual a separação linear passará
    
    - Assim, garantiremos que não haverão interseções entre segmentos de uma mesma envoltória

Embaixo vemos a implementação desse algorimto que verifica se dois segmentos intersectam em quaisquer pontos que não sejam suas extremidades:

In [35]:
def intersectSegments(seg1, seg2):
    # If the segments share an endpoint, they don't "intersect" 
    if (seg1.origin == seg2.origin) or (seg1.origin == seg2.destination) or \
       (seg1.destination == seg2.origin) or (seg1.destination == seg2.destination):
        return False
    
    return seg1.segmentIntersect(seg2)

Agora, vamos escrever alguns testes para esse código:

In [36]:
def test_intersectSegments():
    # Test Case 1: Segments that don't intersect
    seg1 = Segment(Point(0,0), Point(1,1))
    seg2 = Segment(Point(2,2), Point(3,3))
    assert not intersectSegments(seg1, seg2), "Test Case 1 Failed"
    
    # Test Case 2: Segments that intersect
    seg1 = Segment(Point(0,0), Point(2,2))
    seg2 = Segment(Point(2,0), Point(0,2))
    assert intersectSegments(seg1, seg2), "Test Case 2 Failed"
    
    # Test Case 3: Segments that share an endpoint but don't "intersect"
    seg1 = Segment(Point(0,0), Point(2,2))
    seg2 = Segment(Point(2,2), Point(4,4))
    assert not intersectSegments(seg1, seg2), "Test Case 3 Failed"
    
    # Test Case 4: Segments that are colinear but don't overlap
    seg1 = Segment(Point(0,0), Point(2,2))
    seg2 = Segment(Point(3,3), Point(4,4))
    assert not intersectSegments(seg1, seg2), "Test Case 4 Failed"
    
    # Test Case 5: Segments that are colinear and overlap
    seg1 = Segment(Point(0,0), Point(3,3))
    seg2 = Segment(Point(2,2), Point(4,4))
    assert intersectSegments(seg1, seg2), "Test Case 5 Failed"

    # Test Case 6: Vertical segments that don't intersect
    seg1 = Segment(Point(0, 0), Point(0, 2))
    seg2 = Segment(Point(0, 3), Point(0, 5))
    assert not intersectSegments(seg1, seg2), "Test Case 6 Failed"
    
    # Test Case 7: Vertical segments that do intersect
    seg1 = Segment(Point(0, 0), Point(0, 3))
    seg2 = Segment(Point(0, 2), Point(0, 4))
    assert intersectSegments(seg1, seg2), "Test Case 7 Failed"

    print("All test cases passed!")
    
test_intersectSegments()


All test cases passed!


Dado isso, o próximo passo será implementar o algoritmo de varredura linear em si com essa definição atualizada de interseção de segmentos.

Primeiro, vale ressaltar que fizemos uma mudança no algoritmo para tratar de segmentos verticais. Para esses casos, implementamos uma função que desverticaliza um segment vertical infinitezimalmente.

In [37]:
def unverticalize(segment):
    
    if segment.origin.x == segment.destination.x:
        return Segment(segment.destination, Point(segment.origin.x + 1e-9, segment.origin.y))
    
    return segment

Também criamos uma classe para representar os pontos que são considerados eventos para facilitar compreensão e sobrecarregamos seus operadores de comparação para podermos ordená-los

In [38]:
class EventPoint:
    
    def __init__(self, point, segment, isLeft):
        
        self.point = point
        self.segment = segment
        self.isLeft = isLeft

    def __lt__(self, other):

        # The event point with lowest x-coordinate
        # is the lesser event point
        if self.point.x != other.point.x:
            return self.point.x < other.point.x
        
        # If two event points have the same x-coordinate,
        # the event point with lowest y-coordinate is
        # the lesser event point
        if self.point.y != other.point.y:
            return self.point.y < other.point.y
        
        # When two event points coincide and one is a left 
        # endpoint while the other is a right endpoint, the right 
        # endpoint is the lesser event point
        return not self.isLeft and other.isLeft

Aqui, criamos alguns testes para verificarmos a sobrecarga do operador de comparação

In [39]:
def test_event_point_lt():

    # Test x-coordinate priority
    ep1 = EventPoint(Point(1, 2), None, True)
    ep2 = EventPoint(Point(2, 2), None, True)
    assert ep1 < ep2, "Test 1 failed!"
    
    # Test y-coordinate priority
    ep3 = EventPoint(Point(2, 2), None, True)
    ep4 = EventPoint(Point(2, 3), None, True)
    assert ep3 < ep4, "Test 2 failed!"
    
    # Test endpoint type priority
    ep5 = EventPoint(Point(2, 2), None, True)
    ep6 = EventPoint(Point(2, 2), None, False)
    assert ep6 < ep5, "Test 3 failed!"

    # Test with equal x, y, and endpoint type
    ep7 = EventPoint(Point(2, 2), None, True)
    ep8 = EventPoint(Point(2, 2), None, True)
    assert not (ep7 < ep8), "Test 4 failed!"

    print("All tests passed!")

test_event_point_lt()


All tests passed!


Usamos a estrutura de dados "sortedList" da biblioteca sortedContainers para armazenar os segmentos sendo analisados durante a execução do algoritmo. Escolhemos essa estrutura pois:

- Atende aos requisitos de complexidade:
    - Inserção e Remoção em O(log n)
    - Descobrir o sucessor de um segmento conforme a relação de ordem em O(log n)
    - Descobrir o antecessor de um segmento conforme a relação de ordem em O(log n)
- Ampla utilização
- Mantida atual
- Bem documentada
- Otimizado para lidar com grandes volumes de dados

Portanto, segue abaixo o código de varredura linear:

In [40]:
from sortedcontainers import SortedList

def anySegmentIntersect(convexHull1, convexHull2):
    
    allSegments = convexHull1 + convexHull2

    # Create a list of event points
    eventPoints = []

    for segment in allSegments:

        unverticalize(segment)
        
        # Adds the leftmost point of the 
        # segment as an event point
        eventPoints.append(EventPoint(segment.leftmostPoint(), segment, True))
        
        # Adds the rightmost point of the
        # segment as an event point
        if segment.origin == segment.leftmostPoint():
            eventPoints.append(EventPoint(segment.destination, segment, False))
        else:
            eventPoints.append(EventPoint(segment.origin, segment, False))

    # Sort event points in O(nlogn) time
    eventPoints.sort()

    # Create a list of active segments
    # that inserts, removes and searches
    # in O(logn) time
    activeSegments = SortedList()

    for event in eventPoints:
        
        if event.isLeft:  # if it's a left endpoint
            
            activeSegments.add(event.segment)
            
            # Gets the current index of the 
            # current segment in O(logn) time
            idx = activeSegments.index(event.segment)

            # Finds the segment above 
            # (sucessor) in O(log n) time
            segmentAbove = activeSegments[idx + 1] if idx + 1 < len(activeSegments) else None

            # Finds the segment below 
            # (predecessor) in O(log n) time
            segmentBelow = activeSegments[idx - 1] if idx - 1 >= 0 else None

            # Returns true if the segment above
            # or the segment below intersect the
            # current segment
            if segmentAbove and intersectSegments(segmentAbove, event.segment):
                return True
            
            if segmentBelow and intersectSegments(segmentBelow, event.segment):
                return True
            
        else:  # if it's a right endpoint
            
            # Gets the current index of the 
            # current segment in O(logn) time
            idx = activeSegments.index(event.segment)

            # Finds the segment above 
            # (sucessor) in O(log n) time
            segmentAbove = activeSegments[idx + 1] if idx + 1 < len(activeSegments) else None

            # Finds the segment below 
            # (predecessor) in O(log n) time
            segmentBelow = activeSegments[idx - 1] if idx - 1 >= 0 else None

            # Returns true if the segment above
            # and the segment below intersect each other
            if segmentAbove and segmentBelow and intersectSegments(segmentAbove, segmentBelow):
                return True

            activeSegments.remove(event.segment)

    return False

Aqui vamos criar alguns tests para verificar o nosso algoritmo:

In [46]:
def testAnySegmentsIntersect():
    
    # Test 1: Two squares that don't intersect
    segments1 = [
        Segment(Point(0, 0), Point(4, 0)),
        Segment(Point(4, 0), Point(4, 4)),
        Segment(Point(4, 4), Point(0, 4)),
        Segment(Point(0, 4), Point(0, 0))
    ]

    segments2 = [
        Segment(Point(5, 5), Point(9, 5)),
        Segment(Point(9, 5), Point(9, 9)),
        Segment(Point(9, 9), Point(5, 9)),
        Segment(Point(5, 9), Point(5, 5))
    ]

    if anySegmentIntersect(segments1, segments2):
        print("Test 1 failed")


    # Test 2: Two overlapping squares
    segments3 = [
        Segment(Point(2, 2), Point(6, 2)),
        Segment(Point(6, 2), Point(6, 6)),
        Segment(Point(6, 6), Point(2, 6)),
        Segment(Point(2, 6), Point(2, 2))
    ]

    if not anySegmentIntersect(segments1, segments3):
        print("Test 2 failed")


    # Test 3: One square inside another
    segments4 = [
        Segment(Point(1, 1), Point(3, 1)),
        Segment(Point(3, 1), Point(3, 3)),
        Segment(Point(3, 3), Point(1, 3)),
        Segment(Point(1, 3), Point(1, 1))
    ]

    if not anySegmentIntersect(segments1, segments4):
        print("Test 3 failed")


    # Test 4: Triangle and square that intersect at a vertex

    segments9 = [
    Segment(Point(2, 2), Point(4, 2)),
    Segment(Point(4, 2), Point(3, 0)),
    Segment(Point(3, 0), Point(2, 2))
    ]

    segments10 = [
        Segment(Point(3, 0), Point(5, 0)),
        Segment(Point(5, 0), Point(5, 2)),
        Segment(Point(5, 2), Point(3, 2)),
        Segment(Point(3, 2), Point(3, 0))
    ]

    if not anySegmentIntersect(segments9, segments10):
        print("Test 4 failed")


    # Test 5: Pentagon and triangle that don't intersect
    segments11 = [
    Segment(Point(6, 6), Point(8, 6)),
    Segment(Point(8, 6), Point(9, 8)),
    Segment(Point(9, 8), Point(7, 10)),
    Segment(Point(7, 10), Point(5, 8)),
    Segment(Point(5, 8), Point(6, 6))
    ]

    segments12 = [
        Segment(Point(1, 1), Point(3, 1)),
        Segment(Point(3, 1), Point(2, 3)),
        Segment(Point(2, 3), Point(1, 1))
    ]

    if anySegmentIntersect(segments11, segments12):
        print("Test 5 failed")


testAnySegmentsIntersect()

Test 3 failed
Test 4 failed


Veja que o test 4, que é o teste referente a um triangulo que possui um vértice que intercepta o mesmo véritice de um quadrado, falhou mesmo as duas envoltórias se interceptando.

Porém, como havíamos dito anteriormente, isso é exatamente o comportamento que desejávamos pois:

- O vértice onde ocorre a interceptação é extremidade de dois segmentos da mesma envoltória. Não é o caso desse teste mas iríamos querer ignorar essa interceptação de qualquer forma.

- O vértice onde ocorre a interceptação é extremidade de dois segmentos de diferentes envoltórias. Esse é o caso aqui e não tem problema ignorá-lo pois quando gerarmos a separação linear desses dados, a reta passará exatamente sobre esse ponto

Por outro lado, o teste 3, que é o teste referente a uma envoltória contida dentro da outra falhou. Realmente as envoltórias não se interceptam, porém, elas não serão separáveis. Portanto, devemos criar uma nova função que, além de verificar se duas envoltórias se interceptam, também verifica se uma está contida dentro da outra, o que faremos a seguir.

Aqui, criamos alguns testes para nosso algoritmo de varredura linear:

Notes to self:

- lembrar de verificar os casos de exceção do algoritmo de varredura linear:
    - Três segmentos não se interceptam em um único ponto
    - Não há segmentos puramente verticais

# 3. Verificação de separabilidade

# 4. Construção do modelo e classificador

# 5. Método para computação das métricas

# 6. Conclusão dos experimentos

# 7. Conclusão do relatório